In [ ]:
# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

# @noautodeps
# pyre-ignore-all-errors
import json
import logging
import sys

import cloudpickle
from monarch.tools import commands
from example_actors.compute_world_size_actor import ComputeWorldSizeActor
from slurm.utils import get_appdef, get_server_info, create_proc_mesh


logging.basicConfig(
    level=logging.INFO,
    format="%(name)s %(asctime)s %(levelname)s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    force=True,
)


logger: logging.Logger = logging.getLogger(__name__)


async def main():
    num_hosts = 2
    appdef = await get_appdef(num_hosts)
    server_info = await get_server_info(appdef)

    try:
        proc_mesh = await create_proc_mesh(num_hosts, appdef, server_info)
        actor = await proc_mesh.spawn("compute_world_size_actor", ComputeWorldSizeActor)

        logger.info("computing world size...")
        # this is redundant but is here for example sake
        mesh_name = server_info.get_mesh_spec("mesh0").name
        values = await actor.compute_world_size.call(
            master_addr=server_info.host0(mesh_name),
            master_port=29500,
        )

        values_by_rank = {f"rank_{p.rank}": v for p, v in list(values.flatten("rank"))}

        logger.info(
            f"""computed world_sizes:
    {'-'*40}
    {json.dumps(values_by_rank, indent=2)}
    {'-'*40}"""
        )
    finally:
        commands.kill(f"slurm:///{server_info.name}")


if __name__ == "__main__":
    cloudpickle.register_pickle_by_value(sys.modules[ComputeWorldSizeActor.__module__])

    await main()

<string>:8: FutureWarning: Setting `workspace='/home/ubuntu/ahmads/monarch/examples'` is deprecated.
torchx.schedulers.slurm_scheduler 2025-08-29 21:02:20 INFO unable to get job info for `monarch-ubuntu` with `squeue` (squeue: error: Invalid job id: monarch-ubuntu
), trying `sacct`
torchx.schedulers.slurm_scheduler 2025-08-29 21:02:20 INFO unable to get job info for `monarch-ubuntu` with `sacct` (sacct: fatal: Bad job/step specified: monarch-ubuntu
)
monarch.tools.commands 2025-08-29 21:02:20 INFO no existing RUNNING server `slurm:///monarch-ubuntu` creating new one...
torchx.runner.api 2025-08-29 21:02:20 INFO Tracker configurations: {}
torchx.runner.api 2025-08-29 21:02:20 INFO Checking for changes in workspace `/home/ubuntu/.monarch/out/tmpkk97qppi/workspace`...
torchx.runner.api 2025-08-29 21:02:20 INFO To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.
torchx.runner.api 2025-08-29 21:02:20 INFO Reusing original image `monarch_default_workspace:

Ahmad: {'requeue': None, 'ntasks-per-node': '1', 'cpus-per-task': '48', 'mem': '186777', 'gpus-per-task': '4', 'ntasks': '1'}
Ahmad: {'requeue': None, 'ntasks-per-node': '1', 'cpus-per-task': '48', 'mem': '186777', 'gpus-per-task': '4', 'ntasks': '1'}


slurm.utils 2025-08-29 21:02:45 INFO 
===== Server Info =====
{
  "name": "418",
  "server_handle": "slurm:///418",
  "state": "RUNNING",
  "meshes": {
    "mesh0": {
      "host_type": "__UNSET__",
      "hosts": 2,
      "gpus": -1,
      "hostnames": [
        "gpu-queue-st-gpu-compute-1",
        "gpu-queue-st-gpu-compute-2"
      ]
    }
  }
}
__main__ 2025-08-29 21:02:45 INFO computing world size...
monarch._src.actor.allocator 2025-08-29 21:02:45 INFO no match label `procmesh.monarch.meta.com/name` specified in alloc constraints
monarch._src.actor.allocator 2025-08-29 21:02:45 INFO found a single proc mesh `mesh0` in slurm:///418, will allocate on it
monarch.tools.network 2025-08-29 21:02:45 INFO no AF_INET6 address that can bind TCP sockets for `gpu-queue-st-gpu-compute-1:26600` (error: [Errno -5] No address associated with hostname)
monarch.tools.network 2025-08-29 21:02:45 INFO resolved AF_INET address `10.0.2.236:26600` for `gpu-queue-st-gpu-compute-1:26600`
monarch.tools.ne

New job `slurm:///418` is ready to serve.


__main__ 2025-08-29 21:02:53 INFO computed world_sizes:
    ----------------------------------------
    {
  "rank_0": 8,
  "rank_1": 8,
  "rank_2": 8,
  "rank_3": 8,
  "rank_4": 8,
  "rank_5": 8,
  "rank_6": 8,
  "rank_7": 8
}
    ----------------------------------------


>>> Aggregated Logs (2025-08-29 21:02:51) >>>
[8 similar log lines] Initializing process group `nccl`:
[8 similar log lines]   MASTER_ADDR = gpu-queue-st-gpu-compute-1
[8 similar log lines]   MASTER_PORT = 29500
[8 similar log lines]   RANK        = 5
[8 similar log lines]   WORLD_SIZE  = 8
<<< Aggregated Logs (2025-08-29 21:02:54) <<<

